In [1]:
import json
import random
import re
import pathlib
import pandas as pd

In [11]:
dictionary = {}
with open('phoneme-groups-with-syllables.json') as f:
    for l in f:
        j = json.loads(l)
        if len(j['word']) > 1 or j['word'] in ['A', 'I', 'O']:
            dictionary[j['word']] = len(j['syllables'])
        
dict_df = pd.DataFrame.from_records([(k, dictionary[k]) for k in dictionary])
dict_df = dict_df[dict_df[1] <= 7]

In [6]:
jobs_df = pd.read_csv('jobs.csv')
jobs_df.columns

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Code No', 'Level', 'Job Category',
       'Full-Time/Part-Time indicator', 'Salary Range From', 'Salary Range To',
       'Salary Frequency', 'Work Location', 'Division/Work Unit',
       'Job Description', 'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='object')

In [7]:
jobs_df[['Agency', 'Business Title', 'Civil Service Title', 'Job Description']]

,Agency,Business Title,Civil Service Title,Job Description
0,DEPT OF CITYWIDE ADMIN SVCS,Graphic Artist,GRAPHIC ARTIST,"Under the direct supervision, with some latitu..."
1,DEPT OF CITYWIDE ADMIN SVCS,Graphic Artist,GRAPHIC ARTIST,"Under the direct supervision, with some latitu..."
2,DEPARTMENT OF BUSINESS SERV.,Account Manager,CONTRACT REVIEWER (OFFICE OF L,Division of Economic & Financial Opportunity (...
3,DEPARTMENT OF BUSINESS SERV.,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,The New York City Department of Small Business...
4,DEPT OF ENVIRONMENT PROTECTION,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,"Under direct supervision, perform elementary e..."
5,DEPT OF ENVIRONMENT PROTECTION,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,"Under direct supervision, perform elementary e..."
6,DEPT OF ENVIRONMENT PROTECTION,Deputy Plant Chief,SENIOR STATIONARY ENGINEER (EL,"Under general direction, is in responsible cha..."
7,DEPT OF ENVIRONMENT PROTECTION,CIVIL ENGINEERING INTERN,CIVIL ENGINEERING INTERN,The selected candidate will be responsible for...
8,NYC HOUSING AUTHORITY,Temporary Painter,PAINTER,Responsibilities of selected candidates will i...
9,DEPT OF ENVIRONMENT PROTECTION,CIVIL ENGINEERING INTERN,CIVIL ENGINEERING INTERN,The selected candidate will be responsible for...


In [59]:
jobs_df.groupby('Agency').count().sort_values('Job ID', ascending=False)

,Job ID,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
Agency,,,,,,,,,,,,,,,,,,,,,
DEPT OF HEALTH/MENTAL HYGIENE,774,774,774,774,774,774,774,774,774,774,...,774,774,774,774,0,774,774,496,774,774
DEPT OF ENVIRONMENT PROTECTION,536,536,536,536,536,536,536,536,536,536,...,536,536,536,536,0,536,536,12,536,536
DEPARTMENT OF TRANSPORTATION,153,153,153,153,153,153,153,153,153,153,...,153,153,153,153,0,153,153,36,153,153
DEPT OF DESIGN & CONSTRUCTION,125,125,125,125,125,125,125,125,125,125,...,125,125,125,125,0,125,125,10,125,125
DEPT OF INFO TECH & TELECOMM,122,122,122,122,122,122,122,122,122,122,...,122,122,122,122,0,122,122,0,122,122
LAW DEPARTMENT,82,82,82,82,82,82,82,82,82,82,...,82,82,82,82,0,82,82,17,82,82
ADMIN FOR CHILDREN'S SVCS,82,82,82,82,82,82,82,82,82,82,...,82,82,82,82,0,82,82,15,82,82
HOUSING PRESERVATION & DVLPMNT,79,79,79,79,79,79,79,79,79,79,...,79,79,79,79,0,79,79,79,79,79
DEPT OF PARKS & RECREATION,77,77,77,77,77,77,77,77,77,77,...,77,77,77,77,0,77,77,34,77,77


In [60]:
jobs_df.groupby('Business Title').count().sort_values('Job ID', ascending=False)

,Job ID,Agency,Posting Type,# Of Positions,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
Business Title,,,,,,,,,,,,,,,,,,,,,
Project Manager,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,0,26,26,7,26,26
Confidential Investigator,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,0,26,26,26,26,26
College Aide,22,22,22,22,22,22,22,22,22,22,...,22,22,22,22,0,22,22,4,22,22
Assistant Civil Engineer,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,0,18,18,2,18,18
"NFP Nurse Home Visitor, Bureau of Maternal Infant and Reproductive Health",14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,0,14,14,8,14,14
Watershed Maintainer,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,0,14,14,0,14,14
COMPUTER SPECIALIST (SOFTWARE),12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,0,12,12,0,12,12
"Public Health Adviser, Bureau of School Health/SH Nursing Services & Prof Dev",12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,0,12,12,6,12,12
Investigator,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,0,11,11,6,11,11


In [61]:
jobs_df.groupby('Civil Service Title').count().sort_values('Job ID', ascending=False)

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
Civil Service Title,,,,,,,,,,,,,,,,,,,,,
COMPUTER SYSTEMS MANAGER,169,169,169,169,169,169,169,169,169,169,...,169,169,169,169,0,169,169,47,169,169
CITY RESEARCH SCIENTIST,157,157,157,157,157,157,157,157,157,157,...,157,157,157,157,0,157,157,116,157,157
ADMINISTRATIVE PROJECT MANAGER,119,119,119,119,119,119,119,119,119,119,...,119,119,119,119,0,119,119,10,119,119
COMMUNITY COORDINATOR,119,119,119,119,119,119,119,119,119,119,...,119,119,119,119,0,119,119,57,119,119
CLERICAL ASSOCIATE,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,0,90,90,23,90,90
COLLEGE AIDE (ALL CITY DEPTS),78,78,78,78,78,78,78,78,78,78,...,78,78,78,78,0,78,78,48,78,78
CIVIL ENGINEER,58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,0,58,58,3,58,58
PRINCIPAL ADMINISTRATIVE ASSOC,58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,0,58,58,23,58,58
COMMUNITY ASSOCIATE,51,51,51,51,51,51,51,51,51,51,...,51,51,51,51,0,51,51,23,51,51


In [62]:
corpus = []
regex = re.compile(r"[^A-Z\s'-\./]")

#for i,row in jobs_df[['Business Title', 'Civil Service Title', 'Job Description']].iterrows():
for i,row in jobs_df[jobs_df['Civil Service Title'] == 'COMPUTER SYSTEMS MANAGER'][['Job Description']].iterrows():
    #print(i,row)
    words = ' '.join(row)
    #print(words)
    corpus = corpus + [
        i.split() for i in re.split(
            r'[\.!\?;]',regex.sub(
                '', 
                words.upper().replace(",", "").replace("--", " ").replace('(', '').replace(')', '').replace('/', ' ')
            )
        )
    ]
    #print(corpus)
    #break

In [63]:
model2 = {}
for c in corpus:
    for i in range(len(c) - 1):
        w1 = c[i]
        w2 = c[i+1]
        
        if w1 not in model2:
            model2[w1] = {}
            
        if w2 not in model2[w1]:
            model2[w1][w2] = {'count': 1, 'end': 0, 'start': 0}
        else:
            model2[w1][w2]['count'] += 1
            
        if i == (len(c) - 2):
            model2[w1][w2]['end'] += 1
        if i == 0:
            model2[w1][w2]['start'] += 1
            
records = [(w1, w2, model2[w1][w2]['count'], model2[w1][w2]['end'], model2[w1][w2]['start']) for w1 in model2 for w2 in model2[w1]]
model2_df = pd.DataFrame.from_records(records).rename(columns={0: 'word1', 1:'word2', 2:'count', 3: 'end', 4: 'start'})
model2_df = model2_df.merge(dict_df.rename(columns={0: 'word2', 1: 'syllables'}), on='word2')
model2_df = model2_df.merge(dict_df.rename(columns={0: 'word1', 1: 'syllables_word1'}), on='word1')

g = model2_df.groupby('word2')
m = g.sum().reset_index()[['word2', 'end']].merge(g.sum().reset_index()[['word2', 'count']], on='word2')
m['end_percent'] = m['end']/m['count']

model2_df = model2_df.merge(m[['word2', 'end_percent']], on='word2')


g = model2_df.groupby('word1')
m = g.sum().reset_index()[['word1', 'start']].merge(g.sum().reset_index()[['word1', 'count']], on='word1')
m['start_percent'] = m['start']/m['count']

model2_df = model2_df.merge(m[['word1', 'start_percent']], on='word1')

#model2_df.head()

model3 = {}
for c in corpus:
    for i in range(len(c) - 2):
        w1 = c[i]
        w2 = c[i+1]
        w3 = c[i+2]
        
        if w1 not in model3:
            model3[w1] = {}
            
        if w2 not in model3[w1]:
            model3[w1][w2] = {}
            
        if w3 not in model3[w1][w2]:
            model3[w1][w2][w3] = {'count': 1, 'end': 0}
        else:
            model3[w1][w2][w3]['count'] += 1
            
        if i == (len(c) - 3):
            model3[w1][w2][w3]['end'] += 1
            
records = []
for w1 in model3:
    for w2 in model3[w1]:
        for w3 in model3[w1][w2]:
            records.append((w1, w2, w3, model3[w1][w2][w3]['count'], model3[w1][w2][w3]['end']))

model3_df = pd.DataFrame.from_records(records).rename(columns={0: 'word1', 1:'word2', 2: 'word3', 3:'count', 4: 'end'})
model3_df = model3_df.merge(dict_df.rename(columns={0: 'word3', 1: 'syllables'}), on='word3')

g = model3_df.groupby('word3')
m = g.sum().reset_index()[['word3', 'end']].merge(g.sum().reset_index()[['word3', 'count']], on='word3')
m['end_percent'] = m['end']/m['count']

model3_df = model3_df.merge(m[['word3', 'end_percent']], on='word3')

model3_df.head()

,word1,word2,word3,count,end,syllables,end_percent
0,OMD,IS,SEEKING,2,0,2,0.0
1,HPD,IS,SEEKING,2,0,2,0.0
2,SANITATION,IS,SEEKING,4,0,2,0.0
3,TEAM,IS,SEEKING,2,0,2,0.0
4,DIVISION,IS,SEEKING,4,0,2,0.0


In [39]:
for w in jobs_df[jobs_df['Job Description'].str.contains('workshops') & jobs_df['Job Description'].str.contains('activities')]['Job Description']:
    print(w)

The Office of School Health (OSH) is a joint program of the Department of Education and the Department of Health and Mental Hygiene responsible for promoting the health of the 103 million school children enrolled in approximately 1,800 public and non-public schools in New York City. Services to students include case management of chronic health problems, preventive health screening, urgent care, medication administration, preventive counseling, reproductive health services, health education referral for care and assurance of ongoing effective treatment. OSH has been the provider of public health services for 100 years.   DUTIES WILL INCLUDE BUT NOT BE LIMITED TO:   Under the guidance of the Adolescent Health Project Director, the PHA II will:  --Perform the following duties: Assist the Director in the development of reproductive health workshops for high school students.    --Facilitate health education sessions (classroom, small group or individual) to high school student population w

In [43]:
model3_df.sort_values('count')

,word1,word2,word3,count,end,syllables,end_percent
52016,GROUP,SETTING,IN,1,0,1,0.001527
184561,MAINTAINING,CLOSEOUT,CHECKLISTS,1,0,2,0.086957
19180,DAILY,SCHEDULING,OF,1,0,1,0.011718
19179,DAILY,REPORT,OF,1,0,1,0.011718
77704,THE,OCA,TEAM,1,0,1,0.096618
132513,DEMONSTRATIONS,APPROPRIATE,CONDOM,1,0,2,0.000000
19175,DAILY,PERFORMANCE,OF,1,0,1,0.011718
40634,GARAGES,TO,ENSURE,1,0,2,0.001068
158019,DISCIPLINE,FOR,CONFORMITY,1,0,4,0.000000
40637,PROGRAM,DIRECTORS,ENSURE,1,0,2,0.001068


In [46]:
model2_df.sort_values('start_percent')

,word1,word2,count,end,start,syllables,syllables_word1,end_percent,start_percent
0,FRONTS,INCLUDING,4,0,0,3,1,0.000762,0.0
53784,PASSENGER,BOARDING,1,0,0,2,3,0.000000,0.0
53785,PASSENGER,FREIGHT,2,0,0,1,3,0.500000,0.0
53786,PASSENGER,CARS,8,0,0,1,3,0.000000,0.0
53787,ALIGNMENT,AND,2,0,0,1,3,0.000015,0.0
53788,ALIGNMENT,PROVIDE,2,0,0,2,3,0.000000,0.0
53789,ALIGNMENT,WITH,22,0,0,1,3,0.000570,0.0
53790,ALIGNMENT,OF,8,0,0,1,3,0.011742,0.0
53791,ALIGNMENT,TO,4,0,0,1,3,0.018015,0.0
53792,ALIGNMENT,INTEGRATION,2,0,0,4,3,0.024845,0.0


In [69]:
def uppercase(matchobj):
    return matchobj.group(0).upper()

def capitalize(s):
    return re.sub('^([a-z])|[\.|\?|\!]\s*([a-z])|\s+([a-z])(?=\.)', uppercase, s)

#haiku = [[]]
#counts = [5, 7, 5]

def get_first_word():
    subset = model2_df[(model2_df['syllables_word1'] <= 5) & (model2_df['start_percent'] > .1)]
    w = subset.sample(n=1).iloc[0]
    return {'word': w['word1'], 'syllables': w['syllables_word1']}
    #return w#['word2'], w['syllables']

def get_word(previous_words, remaining, line):
    if len(previous_words) >= 2:
        subset = model3_df[
            (model3_df['word1'] == previous_words[-2]['word']) &
            (model3_df['word2'] == previous_words[-1]['word']) & 
            (model3_df['syllables'] <= remaining)
        ]
        
        if line == 2:
            subset = subset[(subset['syllables'] < remaining) | (subset['end_percent'] > .1)]
            
        if len(subset) == 0:
            return get_word([previous_words[-1]], remaining, line)
        
        w = subset.sample(n=1, weights='count').iloc[0]
        
        return {'word': w['word3'], 'syllables': w['syllables']}
    else:
        subset = model2_df[
            (model2_df['word1'] == previous_words[-1]['word']) &
            (model2_df['syllables'] <= remaining)
        ]
        
        if line == 2:
            subset = subset[(subset['syllables'] < remaining) | (subset['end_percent'] > .1)]

        w = subset.sample(n=1, weights='count').iloc[0]

        return {'word': w['word2'], 'syllables': w['syllables']}

def generate_haiku():
    #w = model_df[model_df['syllables']<=5].sample(n=1).iloc[0]
    w = get_first_word()
    #previous_word = w#['word2']
    previous_words = [w]
    haiku = [[w], [], []]
    counts = [5 - w['syllables'], 7, 5]

    for i,l in enumerate(counts):
        remaining = l
        while remaining > 0:
            try:
                w = get_word(previous_words, remaining, i)
                previous_words.append(w)
                haiku[i].append(w)
                remaining -= w['syllables']
            except Exception as e:
                raise e
                previous = haiku[i].pop()
                previous_words.pop()
                remaining += previous['syllables']


    print(capitalize("\n".join([" ".join([w['word'] for w in l]) for l in haiku]).lower()))
    
#generate_haiku()

for i in range(10):
    generated = False
    while not generated:
        try:
            generate_haiku()
            generated = True
        except:
            pass
    print()
#generate_haiku()

Revised status of
agency program and team
to ensure the funds

Develop and or
obtain technical knowledge
of design and skills

The new york city
government more effective
efficient and risks

Plan and schedule for
project program management
security risks

Track all defects and
verify fixes in the
financial systems

The successful its
projects by analyzing
and solving skills sets

Possess and or fee
schedules and leads the planning
and it project plans

These goals the planning
organizing and will work
on all share the funds

During business needs
and meet stakeholders and fit
units and it needs

Gather business rules
user permissions and shape
the team by the needs



https://docs.google.com/forms/d/e/1FAIpQLScHjA-LGbIliFszWZu5CV7c-1JeyZ9R7XozB9kbl1bO1gWRIw/formResponse